In [1]:
#Problem: Predicts if stock price of dow jones will increase or decrease 
#using top news headlines

In [2]:
#import the necessary libraries
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [3]:
#load and store the datasets
df_reddit = pd.read_csv(r"C:\Users\kianh\finsights\finsights_app\finsights_dow_dataset\RedditNews.csv")
df_price = pd.read_csv(r"C:\Users\kianh\finsights\finsights_app\finsights_dow_dataset\upload_DJIA_table.csv")
df_cn = pd.read_csv(r"C:\Users\kianh\finsights\finsights_app\finsights_dow_dataset\Combined_News_DJIA.csv")

In [4]:
#get the merged data set
#merges combined_news and the price data set
df_merge = df_cn.merge(df_price, how='inner', on='Date')

In [5]:
df_reddit.head()

,Date,News
0,2016-07-01,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,IMF chief backs Athens as permanent Olympic host
2,2016-07-01,"The president of France says if Brexit won, so..."
3,2016-07-01,British Man Who Must Give Police 24 Hours' Not...
4,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...


In [6]:
#combine the top news headlines
headlines = []
for row in range(0, len(df_merge.index)):
    headlines.append(' '.join(str(x) for x in df_merge.iloc[row, 2:27]) )

In [7]:
#clean the data(remove b' and b")
clean_headlines = []
for i in range(0, len(headlines)):
    clean_headlines.append(re.sub("b[(')]", '', headlines[i]))
    clean_headlines[i] = re.sub('b[(")]', '',clean_headlines[i])
    clean_headlines[i] = re.sub("\'", '', clean_headlines[i])

In [8]:
pip install datetime

Note: you may need to restart the kernel to use updated packages.


In [9]:
from datetime import datetime
user_date = '2023-07-21'
user_date

'2023-07-21'

In [10]:
df_merge.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top22,Top23,Top24,Top25,Open,High,Low,Close,Volume,Adj Close
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,b'Taliban wages war on humanitarian aid workers',"b'Russia: World ""can forget about"" Georgia\'s...",b'Darfur rebels accuse Sudan of mounting major...,b'Philippines : Peace Advocate say Muslims nee...,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688


In [11]:
#add the clean headlines to the merge data set
df_merge['Combined_News'] = clean_headlines
df_merge['Combined_News'][0]

'Georgia downs two Russian warplanes as countries move to brink of war" BREAKING: Musharraf to be impeached. Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube) Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire Afghan children raped with impunity, U.N. official says - this is sick, a three year old was raped and they do nothing" 150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets. Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SOs side" The enemy combatent trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO] Did the U.S. Prep Georgia for War with Russia? Rice Gives Green Light for Israel to Attack Iran: S

In [12]:
#Polarity = from -1 to 1 to see how positive or negative the statement is
#subjectibity = 0 t 1 on how objective the news article is
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [13]:
#Create 2 new columns
df_merge['Subjectivity'] = df_merge['Combined_News'].apply(getSubjectivity)
df_merge['Polarity'] = df_merge['Combined_News'].apply(getPolarity)

In [14]:
df_merge.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top25,Open,High,Low,Close,Volume,Adj Close,Combined_News,Subjectivity,Polarity
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,"b""No Help for Mexico's Kidnapping Surge""",11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312,Georgia downs two Russian warplanes as countri...,0.267549,-0.048568
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,"b""So this is what it's come to: trading sex fo...",11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609,Why wont America and Nato help us? If they won...,0.374806,0.121956
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727,Remember that adorable 9-year-old who sang at ...,0.536234,-0.044302
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961,U.S. refuses Israel weapons to attack Iran: r...,0.364021,0.011398
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,b'Philippines : Peace Advocate say Muslims nee...,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688,All the experts admit that we should legalise ...,0.375099,0.040677


In [15]:
#Get the sentiment score
#Create a function to get the sentiment scores
def getSIA(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    return sentiment

In [ ]:
#get the sentiment scores for each day
#make this a function that I can use for just one piece of text
compound = []
neg = []
pos = []
neutral = []
SIA = 0

for i in range(0, len(df_merge['Combined_News'])):
    SIA = getSIA(df_merge['Combined_News'][i])
    compound.append(SIA['compound'])
    neg.append(SIA['neg'])
    neutral.append(SIA['neu'])
    pos.append(SIA['pos'])

In [ ]:
#Store the sentiment scores in the merge data frame
df_merge['Compound'] = compound
df_merge['Negative'] = neg
df_merge['Neutral'] = neutral
df_merge['Positive'] = pos

In [ ]:
df_merge.head()

In [ ]:
df_cn.head()

In [ ]:
#create a list of columns to keep
keep_cols = ['Open', 'High', 'Low', 'Volume', 'Subjectivity', 'Polarity', 'Compound', 'Negative', 'Neutral', 'Positive', 'Label']
#keep_cols is not in df_comb, so I need to fix this
df_merge = df_merge[keep_cols]

In [ ]:
df_merge.head()

In [ ]:
#----------RUN THIS--------------------
#create the feature data set
X = df_merge
X = np.array(X.drop(['Label'], 1))
#create the target data set
y = np.array(df_merge['Label'])

In [ ]:
#split the data into 80% training and 20% testing data sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [ ]:
df_cn.head()

In [ ]:
df_merge.head()

In [ ]:
import yfinance as yf

In [ ]:
from datetime import datetime, timedelta

In [ ]:
stock_symbol = 'TSLA'
now = datetime.now()
yesterday = now - timedelta(days=1)
user_date = datetime(yesterday.year, yesterday.month, yesterday.day, hour=10, minute=0, second=0)    
stock_data = yf.download(stock_symbol, start=user_date, end=user_date)

In [ ]:
user_text = "tesla to the moon"
blob = TextBlob(user_text)
sentiment = blob.sentiment

In [ ]:
subjectivity = sentiment.subjectivity
polarity = sentiment.polarity
compound = sentiment.polarity
negative = sentiment.polarity < 0
neutral = sentiment.polarity == 0
positive = sentiment.polarity > 0
label = 'Positive' if sentiment.polarity > 0 else 'Neutral' if sentiment.polarity == 0 else 'Negative'
input_features = [stock_data['Open'].values[0], stock_data['High'].values[0],
                 stock_data['Low'].values[0], stock_data['Volume'].values[0], subjectivity, polarity, compound, negative, neutral, positive, label]

# Create a DataFrame from the list of lists
df = pd.DataFrame([input_features], columns=['Open', 'High', 'Low', 'Volume', 'Subjectivity', 'Polarity', 'Compound', 'Negative', 'Neutral', 'Positive', 'Label'])

In [ ]:
df.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#create and train the model
#this model gets the sentiment
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

#this model gets the forecasted price from the sentiment values
model_price = RandomForestRegressor(random_state=42).fit(x_train, y_train)
# Concatenate the sentiment analysis features with the price features
x_train_combined = np.concatenate((df_merge, df_price), axis=1)

In [ ]:
df_merge.head()

In [ ]:
#need to define y_price and y_sentiment_label
y_price = df_merge['High']  # Assuming the column name for stock prices is 'Stock Price'
y_sentiment_label = df_merge['Label']

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Assume you have prepared the combined feature vector 'X' and the target variables 'y_price' and 'y_sentiment_label'

# Split the data into training and test sets
X_train, X_test, y_price_train, y_price_test, y_sentiment_train, y_sentiment_test = train_test_split(
    X, y_price, y_sentiment_label, test_size=0.2, random_state=42
)

# Regression model for predicting stock price
regression_model = LinearRegression()
regression_model.fit(X_train, y_price_train)
price_predictions = regression_model.predict(X_test)

# Classification model for predicting sentiment label
classification_model = RandomForestClassifier()
classification_model.fit(X_train, y_sentiment_train)
sentiment_label_predictions = classification_model.predict(X_test)

# Now you have two arrays 'price_predictions' and 'sentiment_label_predictions'
# containing the predicted stock prices and sentiment labels, respectively

In [ ]:
y_train

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
x_train_combined

In [ ]:
y_train

In [ ]:
# for col_idx in range(x_train_combined.shape[1]):
#     print(f"Column {col_idx}: Data Type - {x_train_combined[:, col_idx].dtype}")

In [ ]:
# non_numeric_mask = np.vectorize(lambda x: not np.issubdtype(type(x), np.number))(x_train_combined)

# # Select only the rows with numeric values
# x_train_combined_numeric = x_train_combined[~np.any(non_numeric_mask, axis=1)]

In [ ]:
# # Create and train the price prediction model
# price_prediction_model = LinearRegression()
# price_prediction_model.fit(x_train_combined, y_train)

In [ ]:
x_train #x_train is a numpy array

In [ ]:
type(x_test)

In [ ]:
#show the models predictions
predictions = model.predict(x_test)

In [ ]:
predictions[0]

In [ ]:
df_merge.head()

In [ ]:
#so, I need to change my code in vscode so that it takes in all the required info 
#that would be needed to make a prediction
#get the sentiment score

model.predict("dow jones is bullish", datetime.today())

In [ ]:
#show the performance of the model
#0 = negative sentiment
#1 = positive sentiment
#price -> next value i need to implement
print(classification_report(y_test, predictions))

In [ ]:
#serialize into a file
import pickle

In [ ]:
with open('model_pickle', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open('model_pickle', 'rb') as f:
     mp = pickle.load(f)